# The SVM vs. Logistic Regression Showdown

In this lab, you will practice working with non-linear kernels combined with logistic regression and SVM classifiers. The goal is to compare these commonly used techniques. Which comes out on top in terms of accuracy? Runtime? Is there much of a difference at all? Is the dominance of the SVM classifier in machine learning pedagogy justified? 

## Loading the Data

First, we load all the packages we'll need.

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import pairwise_kernels
import scipy
from sklearn import svm, linear_model
from sklearn.model_selection import GridSearchCV
import time

Again we download the data from the Tensorflow package -- See here for [installations instructions](https://www.tensorflow.org/install).  You can get the data from other sources as well. 

In the Tensorflow dataset, the training and test data are represented as arrays:

     Xtr.shape = 60000 x 28 x 28
     Xts.shape = 10000 x 28 x 28
     
The test data consists of `60000` images of size `28 x 28` pixels; the test data consists of `10000` images.

In [3]:
import tensorflow as tf

(Xtr_raw,ytr),(Xts_raw,yts) = tf.keras.datasets.mnist.load_data()

print('Xtr shape: %s' % str(Xtr_raw.shape))
print('Xts shape: %s' % str(Xts_raw.shape))

ntr = Xtr_raw.shape[0]
nts = Xts_raw.shape[0]
nrow = Xtr_raw.shape[1]
ncol = Xtr_raw.shape[2]

Xtr shape: (60000, 28, 28)
Xts shape: (10000, 28, 28)


Each pixel value is from `[0,255]`.  For this lab, it will be convenient to recale the value to -1 to 1 and reshape it as a `ntr x npix` and `nts x npix`.

In [4]:
npix = nrow*ncol
Xtr = (Xtr_raw/255 - 0.5)
Xtr = Xtr.reshape((ntr,npix))

Xts = (Xts_raw/255 - 0.5)
Xts = Xts.reshape((nts,npix))

For this lab we're only going to use a fraction of the MNIST data -- otherwise our models will take too much time and memory to run. Using only part of the training data will of course lead to worse results. Given enough computational resources and time, we would ideally be running on the full data set. The follow code creates a new test and train set, with 5000 examples for train and 4000 for test. 

In [5]:
ntr1 = 5000
nts1 = 4000
Iperm = np.random.permutation(ntr1)
Xtr1 = Xtr[Iperm[:ntr1],:]
ytr1 = ytr[Iperm[:ntr1]]
Iperm = np.random.permutation(nts1)
Xts1 = Xts[Iperm[:nts1],:]
yts1 = yts[Iperm[:nts1]]

## Problem set up and establishing a baseline

To simplify the problem (and speed things up) we're also going to restrict to binary classification. In particular, let's try to design classifier a that separates the 8's from all other digits. To classify intro all 10 classes, we would need to train 10 such classifiers, which would take roughly 10x the computational time. 

Create binary 0/1 label vectors `ytr8` and `yts8` which are 1 wherever `ytr1` and `yts1` equal 8, and 0 everywhere else.

In [6]:
# TODO
# ytr8 = 
# yts8 = 

In [7]:
ytr8 = 1*(ytr1 == 8)
yts8 = 1*(yts1 == 8)

Most of the digits in the test dataset aren't equal to 8. So if we simply guess 0 for every image in `Xts`, we might expect to get classification accuracy around 90%. Our goal should be to significantly beat this **baseline**. 

Formally, write a few lines of code to check what test error would be achieved by the all zeros classifier.

In [8]:
# TODO
# ...
# acc = 
# print('Accuaracy = {0:f}'.format(acc))

In [9]:
acc = 1 - sum(yts8)/yts8.shape[0]
print('Baseline Accuaracy = {0:f}'.format(acc))

Baseline Accuaracy = 0.904000


As a second baseline, let's see how we do with standard (non-kernel) logistic regression. As in the MNIST demo, you can use `scikit-learn`'s built in function `linear_model.LogisticRegression` to fit the model and compute the accuracy. Use no regularization and the `lbfgs` solver. You should acheive an improvement to around 94-95%.

In [10]:
# TODO
# ...
# acc = 
# print('Logistic Regression Accuaracy = {0:f}'.format(acc))

In [11]:
logreg = linear_model.LogisticRegression(verbose=10, solver='lbfgs',max_iter=1000, penalty='none')
logreg.fit(Xtr1,ytr8)
yhat = logreg.predict(Xts1)
acc = np.mean(yhat == yts8)
print('Logistic Regression Accuaracy = {0:f}'.format(acc))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Logistic Regression Accuaracy = 0.919250


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s finished


## Kernel Logistic Regression

To improve on this baseline performance, let's try using the logistic regression classifier with a *non-linear* kernel. Recall from class that any non-linear kernel similarity function $k(\vec{w},\vec{z})$ is equal to $\phi(\vec{w})^T\phi(\vec{z})$ for some feature transformation $\phi$. However, we typically do not need to compute this feature tranformation explicitly: instead we can work directly with the kernel gram matrix $K \in \mathbb{R}^{n\times n}$. Recall that $K_{i,j} = k(\vec{x}_i,\vec{x}_j)$ where $\vec{x}_i$ is the $i^\text{th}$ training data point. 

For this lab we will be using the radial basis function kernel. For a given scaling factor $\gamma$ this kernel is defined as:
$$
k(\vec{w},\vec{z}) = e^{-\gamma\|\vec{w},\vec{z}\|_2^2}
$$

In [12]:
def rbf_kernel(w,z,gamma):
    d = w - z
    return np.exp(-gamma*np.sum(d*d))

Construct the kernel matrix `K1` for `Xtr1` with `gamma = .05`.

In [12]:
# TODO
# K1 = 

In [13]:
gam = 1

In [15]:
n = Xtr1.shape[0]
K1 = np.zeros((n,n))
for i in range(n):
    for j in range(i,n):
        K1[i,j] = rbf_kernel(Xtr1[i,:],Xtr1[j,:],gam)
        K1[j,i] = K1[i,j]

If you used a for loop (which is fine) your code might take several minutes to run! Part of the issue, as we saw with matrix operations in `demo2_more_numpy.ipynb`, is that Python won't know to properly parallize your for loop. For this reason, when constructing kernel matrices it is often faster to us a built-in, carefully optimized function with explicit parallelization. Scikit learn provides such a function through their `metrics` library. 

Referring to the documentation here
https://scikit-learn.org/stable/modules/metrics.html#metrics, use this built in function to recreate the same kernel matrix you did above. Store the result at `K`.

In [15]:
# TODO
# K = 

In [16]:
K = pairwise_kernels(Xtr1, Xtr1, metric='rbf', gamma=gam)

In [15]:
i1 = np.array([[0,0,0,0,0],
               [0,1,1,1,0],
               [0,1,0,1,0],
               [0,1,1,1,0],
               [0,0,0,0,0]])
i2 = np.array([[0,0,0,0,0],
               [0,0,1,0,0],
               [0,0,1,0,0],
               [0,0,1,0,0],
               [0,0,0,0,0]])

In [24]:
i1_vector = i1.flatten()
i2_vector = i2.flatten()

In [25]:
i1_vector

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0])

In [26]:
X = np.vstack((i1_vector,i2_vector))

In [41]:
K = pairwise_kernels(X, metric='rbf', gamma=1/25)

In [42]:
K

array([[1.        , 0.75578374],
       [0.75578374, 1.        ]])

In [18]:
K = pairwise_kernels(Xtr1, metric='rbf', gamma=gam)

In [19]:
K

array([[1.        , 0.04570934, 0.0187686 , ..., 0.01821459, 0.032107  ,
        0.01722649],
       [0.04570934, 1.        , 0.01571971, ..., 0.0153762 , 0.00830409,
        0.00436921],
       [0.0187686 , 0.01571971, 1.        , ..., 0.29203984, 0.00817315,
        0.00712422],
       ...,
       [0.01821459, 0.0153762 , 0.29203984, ..., 1.        , 0.00640406,
        0.01013668],
       [0.032107  , 0.00830409, 0.00817315, ..., 0.00640406, 1.        ,
        0.02495407],
       [0.01722649, 0.00436921, 0.00712422, ..., 0.01013668, 0.02495407,
        1.        ]])

Check that you used the function correctly by writing code to confirm that `K = K1`, or at least that the two are equal up to very small differences (which could arise due to numerical precision issues). Try to do this **without a for loop** for full credit. You will get partial credit if you use a for loop.

In [17]:
# TODO

In [18]:
np.max(np.abs(K1 - K))

1.454392162258955e-14

When using a non-linear kernel, it is important to check that you have chosen reasonable parameters (in our case the only parameter is `gamma`). We typically do not want $k(\vec{x}_i,\vec{x}_j)$ to be either negligably small, or very large for all $i\neq j$ in our data set, or we won't be able to learn anything. For the RBF kernel this means that, for any $\vec{x}_i$ we don't want $k(\vec{x}_i,\vec{x}_j)$ very close to 1 (e.g. .9999) for all $j$, or very close to $0$ (e.g. 1e-8) for all $j$. 

Let's just check that we're in good shape for the first data vector $\vec{x}_0$. Do so by printing out the 10 largest and 10 smallest values of $k(\vec{x}_0,\vec{x}_j)$ for $j\neq 0$. Note that we always have $k(\vec{x}_0,\vec{x}_0) = 1$ for the RBF kernel. 

In [19]:
# TODO
# print('Maximum similarities: \n' + ...)
# print('Minimum similarities: \n' + ...)

In [20]:
vals = np.flip(np.sort(K[0,:]))
print('Maximum similarities: \n' + str(vals[2:12]))
print('Minimum similarities: \n' + str(vals[-10:]))

Maximum similarities: 
[0.0897446  0.08873529 0.0817648  0.07835792 0.07816192 0.07691732
 0.07577733 0.07401104 0.07396206 0.07301938]
Minimum similarities: 
[0.00030576 0.00029968 0.00025396 0.00025231 0.00024373 0.00019725
 0.0001864  0.00016595 0.00012163 0.00010669]


### Implementation
Maybe surprisingly Scikit learn does not have an implementation for kernel logistic regression. To save time, I have implemented my own version for binary classification which uses gradient descent as the optimizer. The function `log_fit` minimizes the $\ell_2$-regularized logisitic regression loss:
$$
L(\vec{\alpha}) =\sum_{i=1}^n (1-y_i)(\phi(\vec{x}_i)^T\phi(\mathbf{X})^T\vec{\alpha}) - \log(h_{\phi(\mathbf{X})^T\vec{\alpha}}(\phi(\vec{x}_i)) + \lambda \|\phi(\vec{X})^T\vec{\alpha}\|_2^2.
$$
As input it takes an $n\times n$ kernel matrix $K$ for the training data, an $n$ length vector `y` of binary class labels, and regularization parameter `lamb`.

**Note:** The version of `grad_opt_adapt` below incorperates a simple automatic stopping rule, which means it will terminate before `nit` if the algorithm seems to have converged.

In [21]:
def grad_opt_adapt(grad_func, beta0, nit=5000, lr_init=1e-3):
    beta = beta0
    lr = lr_init
    L,Lgrad = grad_func(beta0)
    for it in range(nit):
        beta1 = beta - lr*Lgrad
        L1, Lgrad1 = grad_func(beta1)
        df_est = Lgrad.T@(beta1-beta)
        alpha = 0.5
        if (L1-L < alpha*df_est) and (L1 < L):
            lr = lr*2
            L = L1
            Lgrad = Lgrad1
            beta = beta1
        else:
            lr = lr/2 
        if (lr < 1e-15):
            break;
    return beta

# note that I'm taking a slightly different approach here to what we did in `lab_grad_descent`.
# happy discuss if there are any questions. The approach here will lead to faster convergence.
def kernel_grad(alpha,K,y,lamb):
    z = K@alpha
    h = 1/(1+np.exp(-z))
    L = np.sum((1-y)*z - np.log(h)) + lamb*np.sum(z*alpha)

    # Gradient
    Lgrad = (h-y) + 2*lamb*alpha
    return L, Lgrad

def log_fit(K,y,lamb,nit=1000):
    """
    Function which minizes the logistic regression loss 
    """
    kernel_grad_eval = lambda alpha: kernel_grad(alpha,K,y,lamb)
    alpha0 = np.zeros(K.shape[0])
    alpha = grad_opt_adapt(kernel_grad_eval, alpha0, nit=nit, lr_init=1e-5)
    return alpha

Use the `log_fit` function defined above to find parameters `alpha` for the kernel logistic regression model using `lamb = 0` and `K` as constructed above (with `gamma = .05`). You can use the default of 5000 maximum iterations.

In [22]:
# TODO
# alpha = 

In [23]:
nit = 5000
lamb = 0
gam = .05
K = pairwise_kernels(Xtr1,Xtr1,metric='rbf', gamma=gam)
alpha = log_fit(K,ytr8,lamb,nit=nit)

Suppose we have a test dataset with $m$ examples $\vec{w}_1,\ldots, \vec{w}_m$. Once we obtain a coefficient vector $\alpha$, making predictions for any $\vec{w}_j$ in the test set requires computing:
$$
{y}_{j} = \sum_{i=1}^n \alpha_i \cdot k(\vec{w}_{j}, \vec{x}_i).
$$
where $\vec{x}_1, \ldots \vec{x}_n$ are our training data vectors. We classify $\vec{w}_{j}$ in class 0 if ${y}_{j} \leq 0$ and in class 1 if ${y}_{j} > 0$. 

This computation can be rewritten in matrix form as follows:
$$
\vec{y}_{test} = K_{test}\vec{\alpha}, 
$$
where $\vec{y}_{text}$ is an $m$ length vector and $K_{test}$ is a $m\times n$ matrix whose $(j,i)$ entry is equal to $k(\vec{w}_{j}, \vec{x}_i)$. We classify $\vec{w}_{j}$ in class 0 if $\vec{y}_{test}[j] \leq 0$ and in class 1 if $\vec{y}_{test}[j] > 0$. 


Use the `pairwise_kernels` function to construct $K_{test}$. Then make predictions for the test set and evaluate the accuracy of our kernel logistic regression classifier. You should see a pretty substantial lift in accuracy to around $97\%$

In [24]:
# TODO 
# Ktest = ...

In [25]:
# TODO
# yhat = ... (vector containing predicted 0,1 labels)
# acc = np.mean(yhat == yts8)
# print("Test accuracy = %f" % acc)

In [26]:
Ktest = pairwise_kernels(Xts1,Xtr1,metric='rbf', gamma=gam)
yhat = (Ktest@alpha > 0)
acc = np.mean(yhat == yts8)
print("Test accuracy = %f" % acc)

Test accuracy = 0.972000


## Kernel Support Vector Machine

The goal of this lab is to compare Kernel Logistic Regression to Kernel Support Vector machines. Following `demo_mnist_svm.ipynb` implement an SVM classifier using an RBF kernel with `gamma = .05` (the same value we used for logistic regression aboe). Use margin parameter `C = 10`.

In [27]:
# TODO

In [28]:
svc = svm.SVC(probability=False,  kernel="rbf", C=1, gamma=.05,verbose=10)
svc.fit(Xtr1,ytr8)

[LibSVM]

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.05, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=10)

Calculate and print the accuracy of the SVM classifier. You should obtain a similar result as for logistic regression: something close to $97\%$ accuracy. 

In [29]:
# TODO
# ysvm = ... (vector containing predicted 0,1 labels)
# acc = np.mean(yhat == yts8)
# print("Test accuracy = %f" % acc)

In [30]:
ysvm = svc.predict(Xts1)

In [31]:
acc = np.mean(ysvm == yts8)
print("Test accuracy = %f" % acc)

Test accuracy = 0.964500


## The Showdown 

Both SVM classifiers and kernel logisitic regression require tuning parameters to obtain the best possible result. In our setting we will stick with an RBF kernel (although this could be tuned). So we only consider tuning the kernel width parameter `gamma`, as well as the regularization parameter `lamb` for logistic regression, and the margin parameter `C` for SVM. We will choose parameters using for-loops and train-test cross validation.

Train a logistic regression classifier with **all combinations** of the parameters included below in vectors `gamma` and `lamb`. For each setting of parameters, compute and print:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

For computing runtime you might want to use the `time()` function from the `time` library, which we already imported ealier. 

In [32]:
gamma = [.1, .05,.02,.01,.005]
lamb = [0,1e-6,1e-4,1e-2]
# TODO
# ...

In [33]:
gamma = [.1, .05,.02,.01,.005]
lamb = [0,1e-6,1e-4,1e-2]
for g in gamma: 
    for l in lamb:
        t = time.time()
        
        K = pairwise_kernels(Xtr1, Xtr1, metric='rbf', gamma=g)
        Ktest = pairwise_kernels(Xts1,Xtr1,metric='rbf', gamma=g)
        alpha = log_fit(K,ytr8,l)
        yhat = (Ktest@alpha > 0)
        acc = np.mean(yhat == yts8)
        
        elapsed = time.time() - t
        print("gamma = " + str(g) + ", lamb = " + str(l) + ", Test accuracy = %f" % acc + ", Runtime (sec.):  = %f" % elapsed)


gamma = 0.1, lamb = 0, Test accuracy = 0.960000, Runtime (sec.):  = 2.409830
gamma = 0.1, lamb = 1e-06, Test accuracy = 0.959750, Runtime (sec.):  = 3.509907
gamma = 0.1, lamb = 0.0001, Test accuracy = 0.959000, Runtime (sec.):  = 3.074915
gamma = 0.1, lamb = 0.01, Test accuracy = 0.956750, Runtime (sec.):  = 2.197797
gamma = 0.05, lamb = 0, Test accuracy = 0.972000, Runtime (sec.):  = 4.696606
gamma = 0.05, lamb = 1e-06, Test accuracy = 0.969750, Runtime (sec.):  = 8.646823
gamma = 0.05, lamb = 0.0001, Test accuracy = 0.968500, Runtime (sec.):  = 7.613411
gamma = 0.05, lamb = 0.01, Test accuracy = 0.965000, Runtime (sec.):  = 5.619505
gamma = 0.02, lamb = 0, Test accuracy = 0.978750, Runtime (sec.):  = 12.071342
gamma = 0.02, lamb = 1e-06, Test accuracy = 0.979250, Runtime (sec.):  = 11.943973
gamma = 0.02, lamb = 0.0001, Test accuracy = 0.977750, Runtime (sec.):  = 9.488585
gamma = 0.02, lamb = 0.01, Test accuracy = 0.973500, Runtime (sec.):  = 5.970504
gamma = 0.01, lamb = 0, Test a

TODO: What was the best test error achieved, and what setting of parameters achieved this error? Was the kernel logistic regression classifier more sensitive to changes in `gamma` or `lamb`? Discuss in 1-3 short sentences below.

Now let's do the same thing for the kernel Support Vector Classifier. Train an SVM classifier with **all combinations** of the parameters included below in vectors `gamma` and `C`. For each setting of parameters, compute:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

In [34]:
gamma = [.1, .05,.02,.01,.005]
C = [.01,.1,1,10]
# TODO
# ...

In [35]:
gamma = [.1, .05,.02,.01,.005]
C = [1,10,100,1000]
for g in gamma: 
    for c in C:
        t = time.time()
        
        svc = svm.SVC(probability=False,  kernel="rbf", C=c, gamma=g,verbose=10)
        svc.fit(Xtr1,ytr8)
        ysvm = svc.predict(Xts1)
        acc = np.mean(ysvm == yts8)

        elapsed = time.time() - t
        print("gamma = " + str(g) + ", C = " + str(c) + ", Test accuracy = %f" % acc + ", Runtime (sec.):  = %f" % elapsed)

[LibSVM]gamma = 0.1, C = 1, Test accuracy = 0.914500, Runtime (sec.):  = 47.863558
[LibSVM]gamma = 0.1, C = 10, Test accuracy = 0.920000, Runtime (sec.):  = 49.278915
[LibSVM]gamma = 0.1, C = 100, Test accuracy = 0.920000, Runtime (sec.):  = 47.700934
[LibSVM]gamma = 0.1, C = 1000, Test accuracy = 0.920000, Runtime (sec.):  = 47.502115
[LibSVM]gamma = 0.05, C = 1, Test accuracy = 0.964500, Runtime (sec.):  = 22.384640
[LibSVM]gamma = 0.05, C = 10, Test accuracy = 0.967250, Runtime (sec.):  = 23.150993
[LibSVM]gamma = 0.05, C = 100, Test accuracy = 0.967250, Runtime (sec.):  = 23.950929
[LibSVM]gamma = 0.05, C = 1000, Test accuracy = 0.967250, Runtime (sec.):  = 25.652432
[LibSVM]gamma = 0.02, C = 1, Test accuracy = 0.975500, Runtime (sec.):  = 8.616884
[LibSVM]gamma = 0.02, C = 10, Test accuracy = 0.981250, Runtime (sec.):  = 8.712702
[LibSVM]gamma = 0.02, C = 100, Test accuracy = 0.981250, Runtime (sec.):  = 9.241241
[LibSVM]gamma = 0.02, C = 1000, Test accuracy = 0.981250, Runtime (s

TODO: What was the best test error achieved, and what setting of parameters achieved this error? Which performed better in terms of accuracy, the SVM or logisitic regression classifier? How about in terms of runtime?

**NOTE:** For `sklearns`'s built in classifiers, including svm.SVC, there is a function called `GridSearchCV` which can automatically perform hyperparamater tuning for you. The main advantage of the method (as opposed to writing for-loops) is that it supports parallelization, so it can fit with different parameters at the same time. The function also supports automatic $k$-fold cross-validation (instead of simple train/test split). 

You might be interested in using this function for your projects. If so, please check out the tutorial in the following lab from last year: https://github.com/sdrangan/introml/blob/master/unit08_svm/lab_emnist_partial.ipynb.

**20% extra credit:** Find a choice of kernel function (not necessarily Gaussian) and hyperparameters which improves on your best results above by $.5\%$ in test accuracy (for either logistic regression or SVM). You are also free to change the regularization for logistic regression (e.g. to $\ell_1$ regularization), although this will require changing the optimization function. Partial extra credit will be given for small improvements. Report the kernel and hyperparameters used, and compute test accuracy to show your results.